In [72]:
bom_names = ["Copy of WIPRO"]

In [74]:
# Read in the data from the Excel file
import pandas as pd

for bom_name in bom_names:

    df = pd.read_excel(f'{bom_name}.xlsx')

    # Create a dictionary to hold the separate sheets
    sheets_dict = {}

    # Loop through each row in the dataframe
    for index, row in df.iterrows():
        
        # Check if the cell in the first column is not empty
        if pd.notna(row[0]):
            # print(row[0])
            # print(row)
            # Get the item name from the first column
            item_name = row[0]
            
            # Get the exploded items from the second column
            exploded_items = row[3]
            
            # Create a new dataframe with only this row
            new_df = pd.DataFrame([row])
            
            # Add the new dataframe to the dictionary with the item name as the key
            sheets_dict[item_name] = new_df
            
        else:
            
            # Add this row to the existing dataframe in the dictionary
            sheets_dict[item_name] = pd.concat([sheets_dict[item_name], pd.DataFrame([row])])

    # Write each sheet to a new Excel file

    x = 0
    with pd.ExcelWriter(f'{bom_name}name1.xlsx') as writer:
        for item_name, sheet_df in sheets_dict.items():
            x += 1
            # sheet_n = item_name
            # print(item_name[:25] + str(x))
            sheet_n =  sheets_dict[item_name]['Item Name'].iloc[0][:25] + str(x)
            sheet_n = sheet_n.replace('/', '-')
            sheet_df.to_excel(writer, sheet_name=sheet_n, index=False)


    import openpyxl
    from openpyxl.styles import Font

    # Load the workbook
    workbook = openpyxl.load_workbook(f'{bom_name}name1.xlsx')
    print(bom_name)
    # Loop through all the sheets in the workbook
    for sheet in workbook.sheetnames:
        # Select the current sheet
        current_sheet = workbook[sheet]

        from openpyxl.styles import PatternFill
        from openpyxl.utils import get_column_letter

        # Assuming wb is your workbook and sheet is the sheet you want to format
        # Define the colors you want to use for odd and even rocurrent_sheet
        odd_fill = PatternFill(start_color='FFFFFF', end_color='FFFFFF', fill_type='solid')
        even_fill = PatternFill(start_color='B0C4DE', end_color='B0C4DE', fill_type='solid')

        # Iterate over each row in the sheet
        for row in current_sheet.iter_rows():
            # If the row number is odd, apply the odd_fill color
            if row[0].row % 2 == 1:
                for cell in row:
                    cell.fill = odd_fill
            # If the row number is even, apply the even_fill color
            else:
                for cell in row:
                    cell.fill = even_fill
        # Get the max row and column values in the sheet
        max_row = current_sheet.max_row
        max_col = current_sheet.max_column
        # Calculate the total and write it to the sheet
        
        total = sum(current_sheet.cell(row=row, column=max_col).value for row in range(2, max_row+1))
        total_cell = current_sheet.cell(row=max_row+2, column=max_col)
        total_cell.value = "Total"
        total_cell.font = Font(bold=True)
        total_value_cell = current_sheet.cell(row=max_row+3, column=max_col)
        total_value_cell.value = total
        total_value_cell.font = Font(bold=True)


        # Autosize all columns based on their data
        for column in current_sheet.columns:
            column_letter = get_column_letter(column[0].column)
            max_length = 0
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = (max_length + 2) * 1.2
            current_sheet.column_dimensions[column_letter].width = adjusted_width

    # Save the changes to the workbook
    workbook.save(f'{bom_name}autosized.xlsx')



Copy of WIPRO
